In [2]:
import statistics
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skewnorm
import pandas as pd
# from matplotlib.collections import PatchCollection
# from matplotlib.patches import Rectangle
# from matplotlib.markers import MarkerStyle

## Import the logs from .CSV and create DataFrame

In [99]:
logs_df = pd.read_csv('defectRemediation_log_simulation_vPrototype.csv', index_col='ID')
# logs_df['Date'] = pd.to_datetime(logs_df['Date'], format='%Y-%d-%m').dt.date
# logs_df['Timestamp'] = pd.to_datetime(logs_df['Timestamp'], format='%H:%M:%S').dt.time
# logs_df['Date'] = pd.to_datetime(logs_df['Date'], format='%Y-%d-%m')
logs_df['Timestamp'] = pd.to_datetime(logs_df['Timestamp'], format='%Y-%d-%m %H:%M:%S')
logs_df

,Defect_ID,Control_Type,State,Date,Timestamp
ID,,,,,
1,1,ACC03,new,2025-16-05,2025-05-16 00:05:31
2,2,ACC17,new,2025-16-05,2025-05-16 00:12:42
3,3,ACC28,new,2025-16-05,2025-05-16 00:13:44
4,1,ACC03,assign,2025-16-05,2025-05-16 00:18:57
5,1,ACC03,in-progress,2025-16-05,2025-05-16 00:22:43
6,1,ACC03,closed,2025-16-05,2025-05-16 00:34:56
7,2,ACC17,assign,2025-16-05,2025-05-16 00:42:45
8,4,AUTH18,new,2025-16-05,2025-05-16 01:02:12
9,2,ACC17,in-progress,2025-16-05,2025-05-16 01:12:56


### Build incoming defects distributions from logs

In [122]:
# master dictionary with tracked empirical data to build distributions
empirical_dict = {}
control_types = logs_df['Control_Type'].unique()
for control_type in control_types:
    empirical_dict[control_type] = {'incoming_per_hour': [],
                                    'delta_new_assign': [],
                                    'delta_assign_inprogress': [],
                                    'delta_inprogress_closed': []
                                   }

# empirical_dict

In [91]:
hours = list(range(0,24)) + [0] # list of hours from midnight to midnight

# count no° of incoming defects per control type every hour per day WHERE STATE == NEW
for day in logs_df["Date"].unique():
    sub_logs_df = logs_df.query('Date == @day')
    

sub_logs_df

,Defect_ID,Control_Type,State,Date,Timestamp
ID,,,,,
1,1,ACC03,new,2025-05-16,00:05:31
2,2,ACC17,new,2025-05-16,00:12:42
3,3,ACC28,new,2025-05-16,00:13:44
4,1,ACC03,assign,2025-05-16,00:18:57
5,1,ACC03,in-progress,2025-05-16,00:22:43
6,1,ACC03,closed,2025-05-16,00:34:56
7,2,ACC17,assign,2025-05-16,00:42:45
8,4,AUTH18,new,2025-05-16,01:02:12
9,2,ACC17,in-progress,2025-05-16,01:12:56


## Creating Delta table to track time b/w state changes

#### If Delta table already exists, skip the next block; otherwise initialize the empty Delta table:

In [100]:
deltas_df = pd.DataFrame(columns=['Defect_ID','Control_Type','Delta_New_Assign','Delta_Assign_InProgress','Delta_InProgress_Closed'])

In [101]:
try: # deltas_df contains some entries, add onto it
    Last_ID_processed = deltas_df.index[-1]
    index = Last_ID_processed + 1
except IndexError: # deltas_df is empty, start from beginning
    index = 0 

for defect_id in logs_df['Defect_ID'].unique():
    #### Recover timestamps for each state change ####
    timestamp_new = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='new')]['Timestamp'].iloc[0] # OR logs_df.query('(Defect_ID == defect_id) and (State == "new")')['Timestamp'].iloc[0]
    timestamp_assign = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='assign')]['Timestamp'].iloc[0]
    timestamp_inprogress = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='in-progress')]['Timestamp'].iloc[0]
    timestamp_closed = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='closed')]['Timestamp'].iloc[0]

    # time between assign and new (hrs)
    delta_new_assign = timestamp_assign - timestamp_new
    delta_new_assign = round(delta_new_assign.total_seconds()/3600, 3)
    
    # time between in-progress and assign (hrs)
    delta_assign_inprogress = timestamp_inprogress - timestamp_assign
    delta_assign_inprogress = round(delta_assign_inprogress.total_seconds()/3600, 3)
    
    # time between closed and in-progress (hrs)
    delta_inprogress_closed = timestamp_closed - timestamp_inprogress
    delta_inprogress_closed = round(delta_inprogress_closed.total_seconds()/3600, 3)

    # append row to df and iterate index
    control_type = logs_df.loc[logs_df['Defect_ID'] == defect_id].iloc[0]['Control_Type']
    deltas_df.loc[index] = [defect_id, control_type, delta_new_assign, delta_assign_inprogress, delta_inprogress_closed]
    index += 1


In [102]:
deltas_df

,Defect_ID,Control_Type,Delta_New_Assign,Delta_Assign_InProgress,Delta_InProgress_Closed
0,1,ACC03,0.224,0.063,0.204
1,2,ACC17,0.501,0.503,0.779
2,3,ACC28,1.500,1.594,1.067
3,4,AUTH18,1.039,0.229,1.603
4,5,ACC03,0.845,0.466,0.731
5,6,AUTH42,0.378,0.029,0.092
6,7,ACC17,1.597,0.052,0.047


In [57]:
deltas_df.groupby('Control_Type')[['Delta_New_Assign']].mean()

,Delta_New_Assign
Control_Type,
ACC03,0.5345
ACC17,1.0490
ACC28,1.5000
AUTH18,1.0390
AUTH42,0.3780


### Build time b/w state changes distributions from Delta table

In [103]:
deltas_df

,Defect_ID,Control_Type,Delta_New_Assign,Delta_Assign_InProgress,Delta_InProgress_Closed
0,1,ACC03,0.224,0.063,0.204
1,2,ACC17,0.501,0.503,0.779
2,3,ACC28,1.500,1.594,1.067
3,4,AUTH18,1.039,0.229,1.603
4,5,ACC03,0.845,0.466,0.731
5,6,AUTH42,0.378,0.029,0.092
6,7,ACC17,1.597,0.052,0.047


In [125]:
for control_type in control_types:
    empirical_dict[control_type]['delta_new_assign'].extend(deltas_df.query('Control_Type == @control_type')['Delta_New_Assign'].to_list())
    empirical_dict[control_type]['delta_assign_inprogress'].extend(deltas_df.query('Control_Type == @control_type')['Delta_Assign_InProgress'].to_list())
    empirical_dict[control_type]['delta_inprogress_closed'].extend(deltas_df.query('Control_Type == @control_type')['Delta_InProgress_Closed'].to_list())

In [126]:
empirical_dict

{'ACC03': {'incoming_per_hour': [],
  'delta_new_assign': [0.224, 0.845, 0.224, 0.845],
  'delta_assign_inprogress': [0.063, 0.466],
  'delta_inprogress_closed': [0.204, 0.731]},
 'ACC17': {'incoming_per_hour': [],
  'delta_new_assign': [0.501, 1.597, 0.501, 1.597],
  'delta_assign_inprogress': [0.503, 0.052],
  'delta_inprogress_closed': [0.779, 0.047]},
 'ACC28': {'incoming_per_hour': [],
  'delta_new_assign': [1.5, 1.5],
  'delta_assign_inprogress': [1.594],
  'delta_inprogress_closed': [1.067]},
 'AUTH18': {'incoming_per_hour': [],
  'delta_new_assign': [1.039, 1.039],
  'delta_assign_inprogress': [0.229],
  'delta_inprogress_closed': [1.603]},
 'AUTH42': {'incoming_per_hour': [],
  'delta_new_assign': [0.378, 0.378],
  'delta_assign_inprogress': [0.029],
  'delta_inprogress_closed': [0.092]}}

#### Create histograms from empirical_dict